In [23]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import torch
import torchaudio
from torch import nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt
import random
from IPython.display import Audio
from tqdm import tqdm
import seaborn as sns
import pandas as pd
from collections import Counter
from sklearn.manifold import TSNE   # one type of clustering
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.metrics import adjusted_rand_score
from sklearn.metrics import homogeneity_score, completeness_score, v_measure_score
from itertools import combinations
from sklearn.decomposition import PCA
from scipy.linalg import block_diag
import pickle
from scipy import stats
from scipy.interpolate import interp1d
from scipy.stats import sem
import argparse

from model_padding import generate_mask_from_lengths_mat, mask_it
from paths import *
from misc_my_utils import *
from model_loss import *
from model_model import VQVAEV1 as TheLearner
from model_dataset import TargetDatasetBoundary as ThisDataset
from model_dataset import Normalizer, DeNormalizer, TokenMap
from model_dataset import MelSpecTransformDB as TheTransform
from model_dataset import DS_Tools
from reshandler import DictResHandler
from misc_progress_bar import draw_progress_bar
from test_bnd_detect_tools import *
from misc_tools import PathUtils as PU
from misc_tools import AudioCut, ARPABET
from misc_my_utils import time_to_frame

# Constants
EPOCHS = 10
BATCH_SIZE = 1
INPUT_DIM = 64
OUTPUT_DIM = 64
INTER_DIM_0 = 32
INTER_DIM_1 = 16
INTER_DIM_2 = 8
ENC_SIZE_LIST = [INPUT_DIM, INTER_DIM_0, INTER_DIM_1, INTER_DIM_2]
DEC_SIZE_LIST = [OUTPUT_DIM, INTER_DIM_0, INTER_DIM_1, INTER_DIM_2]
DROPOUT = 0.5
NUM_LAYERS = 2
EMBEDDING_DIM = 128
REC_SAMPLE_RATE = 16000
N_FFT = 400
N_MELS = 64
LOADER_WORKER = 16

In [24]:

def cutHid(hid, cutstart, cutend, start_offset=0, end_offset=1): 
    selstart = max(cutstart, int(cutstart + (cutend - cutstart) * start_offset))
    selend = min(cutend, int(cutstart + (cutend - cutstart) * end_offset))
    # hid is (L, H)
    return hid[selstart:selend, :]

# we have very limited data, so we don't need to select, just plot all
def get_toplot(hiddens, sepframes, phi_types, stop_names, offsets=(0, 1)): 
    cutstarts = []
    cutends = []
    for hidden, sepframe, phi_type in zip(hiddens, sepframes, phi_types):
        if phi_type == 'ST':
            cutstarts.append(sepframe)
        else:
            cutstarts.append(0)
        cutends.append(hidden.shape[0])
    
    hid_sel = np.empty((0, 8))
    tag_sel = []
    for (item, start, end, tag) in zip(hiddens, cutstarts, cutends, phi_types): 
        hid = cutHid(item, start, end, offsets[0], offsets[1])
        hidlen = hid.shape[0]
        hid_sel = np.concatenate((hid_sel, hid), axis=0)
        tag_sel += [tag] * hidlen
    return hid_sel, np.array(tag_sel)




In [25]:

def main(train_name, ts, stop_epoch, guide_dir, prefix=""):
    # parser = argparse.ArgumentParser(description='argparse')
    # parser.add_argument('--timestamp', '-ts', type=str, default="0000000000", help="Timestamp for project")
    # parser.add_argument('--epoch','-e',type=str, default=0, help='The epoch to load')
    # parser.add_argument('--gpu', '-gpu', type=int, default=0, help="Choose the GPU to work on")
    # args = parser.parse_args()

    # Dirs
    rec_dir = train_cut_phone_
    # Prepare model paths
    model_save_dir = os.path.join(model_save_, f"{train_name}-{ts}")
    res_save_dir = os.path.join(model_save_dir, "eval_res")
    assert PU.path_exist(model_save_dir)
    mk(res_save_dir)
    print(f"{train_name}-{ts} at epoch {stop_epoch} is being evaluated")

    mytrans = TheTransform(sample_rate=REC_SAMPLE_RATE, 
                        n_fft=N_FFT, n_mels=N_MELS, 
                        normalizer=Normalizer.norm_mvn, 
                        denormalizer=DeNormalizer.norm_mvn)

    st_valid = pd.read_csv(os.path.join(guide_dir, prefix + "ST-valid.csv"))
    t_valid = pd.read_csv(os.path.join(guide_dir, prefix + "T-valid.csv"))
    t_valid["sibilant_startTime"] = 0
    all_valid = pd.concat([t_valid, st_valid], ignore_index=True, sort=False)

    valid_ds = ThisDataset(rec_dir, 
                        all_valid, 
                        transform=mytrans)

    valid_loader = DataLoader(valid_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=LOADER_WORKER, collate_fn=ThisDataset.collate_fn)
    valid_num = len(valid_loader.dataset)

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    recon_loss = nn.MSELoss(reduction='none')
    masked_recon_loss = MaskedLoss(recon_loss)
    model_loss = masked_recon_loss

    # model = TheLearner(enc_size_list=ENC_SIZE_LIST, dec_size_list=DEC_SIZE_LIST, num_layers=2)
    model = TheLearner(enc_size_list=ENC_SIZE_LIST, 
                    dec_size_list=DEC_SIZE_LIST, 
                    embedding_dim=EMBEDDING_DIM, 
                    num_layers=NUM_LAYERS, dropout=DROPOUT)
    # model = TheLearner(enc_size_list=ENC_SIZE_LIST, dec_size_list=DEC_SIZE_LIST, num_layers=1)
    model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-3)

    model_name = "{}.pt".format(stop_epoch)
    model_path = os.path.join(model_save_dir, model_name)
    state = torch.load(model_path)

    model.load_state_dict(state)
    model.to(device)

    model.eval()
    reshandler = DictResHandler(whole_res_dir=res_save_dir, 
                                 file_prefix=f"hidrep-{stop_epoch}")
    all_ze = []
    all_zq = []
    # all_name = []
    all_stop_names = []
    all_sepframes = []
    all_phi_type = []

    for (x, x_lens, pt, sn, sf) in tqdm(valid_loader): 
        # name = name[0]

        x_mask = generate_mask_from_lengths_mat(x_lens, device=device)
        
        x = x.to(device)

        x_hat, attn_w, (ze, zq) = model(x, x_lens, x_mask)

        ze = ze.cpu().detach().numpy().squeeze()
        zq = zq.cpu().detach().numpy().squeeze()

        all_ze += [ze]
        all_zq += [zq]
        all_stop_names += sn
        all_sepframes += sf
        all_phi_type += pt
    
    reshandler.res["ze"] = all_ze
    reshandler.res["zq"] = all_zq
    reshandler.res["sn"] = all_stop_names
    reshandler.res["sep-frame"] = all_sepframes
    reshandler.res["phi-type"] = all_phi_type
    reshandler.save()
    print(f"Results saved to {res_save_dir}")

    
    cluster_groups = ["T", "ST"]

    hidr_cs, tags_cs = get_toplot(hiddens=all_zq, 
                                    sepframes=all_sepframes,
                                    phi_types=all_phi_type,
                                    stop_names=all_stop_names,
                                    offsets=(0.4, 0.6))
    color_translate = {item: idx for idx, item in enumerate(cluster_groups)}
    # Use Counter to count the occurrences of each item
    # item_counts = Counter(tags_cs)
    X, Y = hidr_cs, tags_cs
    silhouette_avg = silhouette_score(X, tags_cs)
    
    # n_clusters = len(cluster_groups)
    # kmeans = KMeans(n_clusters=n_clusters, random_state=0)
    # cluster_labels = kmeans.fit_predict(X)

    # silhouette_avg = adjusted_rand_score(tags_cs, cluster_labels)
    return silhouette_avg

In [26]:
if __name__ == "__main__":
    train_name = "C_01"
    ts = "0226154925"
    sil_list = []
    guide_dir = src_
    for i in range(0, 40):
        sil_score = main(train_name, ts, i, guide_dir, prefix="phi-")
        sil_list.append(sil_score)

    model_save_dir = os.path.join(model_save_, f"{train_name}-{ts}")
    res_save_dir = os.path.join(model_save_dir, "eval_res")
    plt.plot(sil_list) # Plot the points
    plt.title(f'Middle Silhouette Score')
    plt.xlabel('Epoch')
    plt.ylabel('Silhouette Score')
    # plt.show()
    plt.savefig(os.path.join(res_save_dir, f"silplot-global.png"))
    plt.close()


    train_name = "C_02"
    ts = "0301171500"
    sil_list = []
    guide_dir = os.path.join(src_, "phi-unbalanced-guides")
    for i in range(0, 40):
        sil_score = main(train_name, ts, i, guide_dir, prefix="")
        sil_list.append(sil_score)

    model_save_dir = os.path.join(model_save_, f"{train_name}-{ts}")
    res_save_dir = os.path.join(model_save_dir, "eval_res")
    plt.plot(sil_list) # Plot the points
    plt.title(f'Middle Silhouette Score')
    plt.xlabel('Epoch')
    plt.ylabel('Silhouette Score')
    # plt.show()
    plt.savefig(os.path.join(res_save_dir, f"silplot-global.png"))
    plt.close()

C_01-0226154925 at epoch 0 is being evaluated


  0%|          | 0/1116 [00:00<?, ?it/s]

100%|██████████| 1116/1116 [00:06<00:00, 175.38it/s]


Results saved to ../model_save/C_01-0226154925/eval_res
C_01-0226154925 at epoch 1 is being evaluated


100%|██████████| 1116/1116 [00:08<00:00, 126.14it/s]


...

Results saved to ../model_save/C_02-0301171500/eval_res
C_02-0301171500 at epoch 38 is being evaluated


100%|██████████| 2617/2617 [00:13<00:00, 188.49it/s]


Results saved to ../model_save/C_02-0301171500/eval_res
C_02-0301171500 at epoch 39 is being evaluated


100%|██████████| 2617/2617 [00:12<00:00, 201.42it/s]


Results saved to ../model_save/C_02-0301171500/eval_res
